# 4. Defining new Physics-Based Prognostic Models

All of the past sections describe how to use an existing model. In this section we will describe how to create a new model. This section specifically describes creating a new physics-based model. For training and creating data-driven models see 5. Data-driven Models.

## Linear Models

The easiest model to build is a linear model. Linear models are defined as a linear time series, which can be defined by the following equations:



**The State Equation**:
$$
\frac{dx}{dt} = Ax + Bu + E
$$

**The Output Equation**:
$$
z = Cx + D
$$

**The Event State Equation**:
$$
es = Fx + G
$$

$x$ is `state`, $u$ is `input`, $z$ is `output`, and $es$ is `event state`

Linear Models are defined by creating a new model class that inherits from progpy's LinearModel class and defines the following properties:
* $A$: 2-D np.array[float], dimensions: n_states x n_states. <font color = 'teal'>The state transition matrix. It dictates how the current state affects the change in state dx/dt.</font>
* $B$: 2-D np.array[float], optional (zeros by default), dimensions: n_states x n_inputs. <font color = 'teal'>The input matrix. It dictates how the input affects the change in state dx/dt.</font>
* $C$: 2-D np.array[float], dimensions: n_outputs x n_states. The output matrix. <font color = 'teal'>It determines how the state variables contribute to the output.</font>
* $D$: 1-D np.array[float], optional (zeros by default), dimensions: n_outputs x 1. <font color = 'teal'>A constant term that can represent any biases or offsets in the output.</font>
* $E$: 1-D np.array[float], optional (zeros by default), dimensions: n_states x 1. <font color = 'teal'>A constant term, representing any external effects that are not captured by the state and input.</font>
* $F$: 2-D np.array[float], dimensions: n_es x n_states. <font color = 'teal'>The event state matrix, dictating how state variables contribute to the event state.</font>
* $G$: 1-D np.array[float], optional (zeros by default), dimensions: n_es x 1. <font color = 'teal'>A constant term that can represent any biases or offsets in the event state.</font>
* __inputs__:  list[str] - `input` keys
* __states__:  list[str] - `state` keys
* __outputs__: list[str] - `output` keys
* __events__:  list[str] - `event` keys

We will now utilize our LinearModel to model the classical physics problem throwing an object into the air. This is a common example model, the non-linear version of which (`progpy.examples.ThrownObject`) has been used frequently throughout the examples. This version of ThrownObject will behave nearly identically to the non-linear ThrownObject, except it will not have the non-linear effects of air resistance.

We can create a subclass of LinearModel which will be used to simulate an object thrown, which we will call the ThrownObject Class.

First, some definitions for our Model:

**Events**: (2)
* `falling: The object is falling`
* `impact: The object has hit the ground`

**Inputs/Loading**: (0)
* `None`

**States**: (2)
* `x: Position in space (m)`
* `v: Velocity in space (m/s)`

**Outputs/Measurements**: (1)
* `x: Position in space (m)`

Now, for our keyword arguments:

* <font color = green>__thrower_height : Optional, float__</font>
  * Height of the thrower (m). Default is 1.83 m
* <font color = green>__throwing_speed : Optional, float__</font>
  * Speed at which the ball is thrown (m/s). Default is 40 m/s

With our definitions, we can now create the ThrownObject Model.

First, we need to import the necessary packages.

In [ ]:
import numpy as np
from progpy import LinearModel

Now we'll define some features of a ThrownObject LinearModel. Recall that all LinearModels follow a set of core equations and require some specific properties (see above). In the next step, we'll define our inputs, states, outputs, and events, along with the $A$, $C$, $E$, and $F$ values.

First, let's consider state transition. For an object thrown into the air without air resistance, velocity would decrease literally by __-9.81__ 
$\dfrac{m}{s^2}$ due to the effect of gravity, as described below:

 $$\frac{dv}{dt} = -9.81$$

 Position change is defined by velocity (v), as described below:
 
 $$\frac{dx}{dt} = v$$

 Note: For the above equation x is position not state. Combining these equations with the model $\frac{dx}{dt}$ equation defined above yields the A and E matrix defined below. Note that there is no B defined because this model does not have any inputs.

In [ ]:
class ThrownObject(LinearModel):
    events = ['impact']
    inputs = []  
    states = ['x', 'v']
    outputs = ['x']
    
    A = np.array([[0, 1], [0, 0]])
    C = np.array([[1, 0]])
    E = np.array([[0], [-9.81]])
    F = None

Note that we defined our `A`, `C`, `E`, and `F` values to fit the dimensions that were stated at the beginning of the notebook! Since the parameter `F` is not optional, we have to explicitly set the value as __None__.

Next, we'll define some default parameters for our ThrownObject model.

In [ ]:
class ThrownObject(ThrownObject):  # Continue the ThrownObject class
    default_parameters = {
        'thrower_height': 1.83,
        'throwing_speed': 40,
    }

In the following cells, we'll define some class functions necessary to perform prognostics on the model.

The `initialize()` function sets the initial system state. Since we have defined the `x` and `v` values for our ThrownObject model to represent position and velocity in space, our initial values would be the thrower_height and throwing_speed parameters, respectively.

In [ ]:
class ThrownObject(ThrownObject):
    def initialize(self, u=None, z=None):
        return self.StateContainer({
            'x': self.parameters['thrower_height'],
            'v': self.parameters['throwing_speed']
            })

For our `threshold_met()`, we define the function to return True for event 'falling' when our thrown object model has a velocity value of less than 0 (object is 'falling') and for event 'impact' when our thrown object has a distance from of the ground of less than or equal to 0 (object is on the ground, or has made 'impact').

`threshold_met()` returns a _dict_ of values, if each entry of the _dict_ is __True__, then our threshold has been met!

In [ ]:
class ThrownObject(ThrownObject):
    def threshold_met(self, x):
        return {
            'falling': x['v'] < 0,
            'impact': x['x'] <= 0
        }

Finally, for our `event_state()`, we will calculate the measurement of progress towards the events. We normalize our values such that they are in the range of 0 to 1, where 0 means the event has occurred.

In [ ]:
class ThrownObject(ThrownObject):
    def event_state(self, x): 
        x_max = x['x'] + np.square(x['v'])/(9.81*2)
        return {
            'falling': np.maximum(x['v']/self.parameters['throwing_speed'],0),
            'impact': np.maximum(x['x']/x_max,0) if x['v'] < 0 else 1
        }

With these functions created, we can now use the `simulate_to_threshold()` function to simulate the movement of the thrown object in air. For more information, see 1. Simulation.

In [ ]:
m = ThrownObject()
save = m.simulate_to_threshold(print=True, save_freq=1, threshold_keys='impact', dt=0.1)

__Note__: Because our model takes in no inputs, we have no need to define a future loading function. However, for most models, there would be inputs, and thus a need for a future loading function. For more information on future loading functions and when to use them, please refer to the future loading section in 1. Simulation.

Let's take a look at the outputs of this model

In [ ]:
fig = save.outputs.plot(title='generated model')

Notice that that plot resembles a parabola, which represents the position of the ball through space as time progresses!

For more information on Linear Models, see the [Linear Model](https://nasa.github.io/progpy/api_ref/prog_models/LinearModel.html) Documentation.

## New State Tranisition Models

## Direct Models

## Derived Parameters

## Matrix Models

## State Limits

## Custom Events

## Serialization 

ProgPy includes a feature to serialize models, which we highlight in this section. 

Model serialization has a variety of purposes. For example, serialization allows us to save a specific model to a file to be loaded later, or can aid us in sending a model to another machine over a network connection. In this section, we'll show how to serialize and deserialize model objects using `pickle` and `JSON` methods. 

First, we'll import the necessary modules.

In [ ]:
import matplotlib.pyplot as plt
import pickle
import numpy as np
from progpy.models import BatteryElectroChemEOD

For this example, we'll use the BatteryElectroChemEOD model. We'll start by creating a model object. 

In [ ]:
batt = BatteryElectroChemEOD()

In order to illustrate a perfect match between the original and serialized versions of the model, we'll set both the `process_noise` and `measurement_noise` to 0. 

In [ ]:
batt.parameters['process_noise'] = 0
batt.parameters['measurement_noise'] = 0

First, we'll serialize the model in two different ways using 1) `pickle` and 2) `JSON`. Then, we'll plot the deserialized results to show equivalence of the methods. 

To save using the `pickle` package, we'll serialize the model using the `dump` method. Once saved, we can then deserialize using the `load` method. In practice, deserializing will likely occur in a different file or in a later use-case, but here we deserialize to show equivalence of the saved model. 

In [ ]:
pickle.dump(batt, open('save_pkl.pkl', 'wb')) # Serialize model
load_pkl = pickle.load(open('save_pkl.pkl', 'rb')) # Deserialize model 

Next, we'll serialize using the `to_json` method. We deserialize by calling the model directly with the serialized result using the `from_json` method.

In [ ]:
save_json = batt.to_json() # Serialize model
json_1 = BatteryElectroChemEOD.from_json(save_json) # Deserialize model

Note that the serialized result can also be saved to a text file and uploaded for later use. We demonstrate this below:

In [ ]:
txtFile = open("save_json.txt", "w")
txtFile.write(save_json)
txtFile.close()

with open('save_json.txt') as infile: 
    load_json = infile.read()

json_2 = BatteryElectroChemEOD.from_json(load_json)

We have now serialized and deserialized the model using `pickle` and `JSON` methods. Let's compare the resulting models. To do so, we'll use ProgPy's [simulation](https://nasa.github.io/progpy/prog_models_guide.html#simulation) to simulate the model to threshold and compare the results. 

First, we'll need to define our [future loading profile](https://nasa.github.io/progpy/prog_models_guide.html#future-loading).

In [ ]:
def future_loading(t, x=None):
    if (t < 600):
        i = 3
    elif (t < 1000):
        i = 2
    elif (t < 1500):
        i = 1.5
    else:
        i = 4
    return batt.InputContainer({'i': i})

Now, let's simulate each model to threshold using the `simulate_to_threshold` method. 

In [ ]:
# Original model 
results_orig = batt.simulate_to_threshold(future_loading, save_freq = 1)
# Pickled version  
results_pkl = load_pkl.simulate_to_threshold(future_loading, save_freq = 1)
# JSON versions
results_json_1 = json_1.simulate_to_threshold(future_loading, save_freq = 1)
results_json_2 = json_2.simulate_to_threshold(future_loading, save_freq = 1)


Finally, let's plot the results for comparison.

In [ ]:
voltage_orig = [results_orig.outputs[iter]['v'] for iter in range(len(results_orig.times))]
voltage_pkl = [results_pkl.outputs[iter]['v'] for iter in range(len(results_pkl.times))]
voltage_json_1 = [results_json_1.outputs[iter]['v'] for iter in range(len(results_json_1.times))]
voltage_json_2 = [results_json_2.outputs[iter]['v'] for iter in range(len(results_json_2.times))]

plt.plot(results_orig.times,voltage_orig,'-b',label='Original surrogate') 
plt.plot(results_pkl.times,voltage_pkl,'--r',label='Pickled serialized surrogate') 
plt.plot(results_json_1.times,voltage_json_1,'-.g',label='First JSON serialized surrogate') 
plt.plot(results_json_2.times, voltage_json_2, '--y', label='Second JSON serialized surrogate')
plt.legend()
plt.xlabel('Time (sec)')
plt.ylabel('Voltage (volts)')

All of the voltage curves overlap, showing that the different serialization methods produce the same results. 

Additionally, we can compare the output arrays directly, to ensure equivalence. 

In [ ]:
import numpy as np

# Check if the arrays are the same
are_arrays_same = np.array_equal(voltage_orig, voltage_pkl) and \
                  np.array_equal(voltage_orig, voltage_json_1) and \
                  np.array_equal(voltage_orig, voltage_json_2)

print(f"The simulated results from the original and serialized models are {'identical. This means that our serialization works!' if are_arrays_same else 'not identical. This means that our serialization does not work.'}!")

To conclude, we have shown how to serialize models in ProgPy using both `pickle` and `JSON` methods. Understanding how to serialize and deserialize models can be a powerful tool for prognostics developers. It enables the saving of models to a disk and the re-loading of these models back into memory at a later time. 

## Conclusions